In [ ]:
import numpy as np
import training_data_generation as tdg
import matplotlib.pyplot as plt
import matplotlib as mpl

%matplotlib inline
plt.ion();

#plt.style.use('classic')
fontsize=22
params = {
  'font.size' : fontsize,
  #'axes.titlesize' : fontsize,
  #'axes.labelsize': fontsize,
  #'legend.fontsize': fontsize,
  #'xtick.labelsize': fontsize,
  #'ytick.labelsize': fontsize,
  'text.usetex': True,
  'axes.unicode_minus': True}
plt.rcParams.update(params)
plt.rc('font', **{'family':'serif', 'serif':['Computer Modern Roman'],
                        'monospace': ['Computer Modern Typewriter']})

In [ ]:
import healpy as hp
import utils_intensity_map as uim
import utils_deck_generation as idg
import netcdf_read_write as nrw
import training_data_generation as tdg
import utils_healpy as uhp
import os
%matplotlib inline
np_complex = np.vectorize(complex)

In [ ]:
num_comparisons = 2

diag_dir = [None] * num_comparisons
iex = [None] * num_comparisons
sys_params = [None] * num_comparisons
run_location = [None] * num_comparisons

In [ ]:
plot_labels = ["N190204_003","Optimized beam parameters"]
max_mode = 25
modes_xlabel = "l mode"
modes_ylabel = r"Mode amplitude ($\%$)"

In [ ]:
diag_dir[0] = "Data_230802a_N190204_003_4ns_1_3_weighting"
iex[0] = 0
sys_params[0] = tdg.define_system_params(diag_dir[0])
run_location[0] = sys_params[0]["root_dir"] + "/" + sys_params[0]["sim_dir"] + str(iex[0])

diag_dir[1] = "Data_230802a_N190204_003_4ns_1_3_weighting"
iex[1] = "517_scaled_new"
sys_params[1] = tdg.define_system_params(diag_dir[1])
run_location[1] = sys_params[1]["root_dir"] + "/" + sys_params[1]["sim_dir"] + str(iex[1])

with_pointing_markers = True
import_flipped = False
old_format = True
display_steradians = False

## Initial intensity modes

In [ ]:
dataset_params = [None] * num_comparisons
facility_spec = [None] * num_comparisons
intensity_map_sr = [None] * num_comparisons
intensity_map = [None] * num_comparisons
parameters = [None] * num_comparisons
intensity_map_normalized = [None] * num_comparisons
avg_flux = [None] * num_comparisons
rms = [None] * num_comparisons
power_spectrum_unweighted = [None] * num_comparisons


In [ ]:
for ind in range(num_comparisons):
    num_examples = 1
    sys_params[ind]["run_gen_deck"] = False

    if old_format:
        dataset_params[ind], facility_spec[ind] = tdg.define_dataset_params(int(num_examples))
    else:
        dataset_params[ind] = nrw.read_general_netcdf(diag_dir[ind] + "/" + sys_params[ind]["dataset_params_filename"])
        facility_spec[ind] = nrw.read_general_netcdf(diag_dir[ind] + "/" + sys_params[ind]["facility_spec_filename"])
    dataset_params[ind]["num_examples"] = num_examples

    if old_format:
        intensity_map_sr[ind] = nrw.read_intensity(run_location[ind], dataset_params[ind]["imap_nside"], facility_spec[ind]['target_radius'])
        # convert from W/sr to W/cm**2
        intensity_map[ind] = intensity_map_sr[ind] / (facility_spec[ind]['target_radius'] / 10000.0)**2
    else:
        parameters[ind] = nrw.read_general_netcdf(run_location[ind] + "/" + sys_params[ind]["ifriit_ouput_name"])
        intensity_map[ind] = parameters[ind]["intensity"]
        # convert from W/cm**2 to W/sr
        intensity_map_sr[ind] = parameters[ind]["intensity"] * (facility_spec[ind]['target_radius'] / 10000.0)**2

    intensity_map_normalized[ind], avg_flux[ind] = uim.imap_norm(intensity_map[ind])
    rms[ind] = uim.extract_rms(intensity_map_normalized[ind])
    
    LMAX = dataset_params[ind]["LMAX"]
    power_spectrum_unweighted[ind] = uhp.power_spectrum(intensity_map[ind], LMAX)


In [ ]:
aspc = 1.2
fig1 = plt.figure(num=1, figsize=(6*aspc,6), facecolor='white')

nside = 64
npix = hp.nside2npix(nside)
#vec = hp.ang2vec(np.pi, np.pi * 3 / 4)
vec = hp.ang2vec(np.pi/2, np.pi * 1/2)
ipix_disc = hp.query_disc(nside=nside, vec=vec, radius=np.radians(90))
masked_intensity_map = intensity_map[0]
masked_intensity_map[ipix_disc] = intensity_map[1][ipix_disc]

projection = hp.mollview(masked_intensity_map, unit=r"$\rm{W/cm^2}$", title="", flip="geo", return_projected_map=True)
hp.graticule()

In [ ]:

aspc = 2.0
fig2 = plt.figure(num=2, figsize=(6*aspc,6), facecolor='white')
ax2 = fig2.add_axes([0.15, 0.14, 0.55, 0.6])
cax1 = fig2.add_axes([0.75, 0.14, 0.02, 0.6])

cmesh = ax2.pcolormesh(projection, linewidth=0, rasterized=True)
cbar = fig2.colorbar(cmesh, ax=[ax2], cax=cax1)
ax2.axis('off')

cbar.set_label("Intensity" + r" $(W/cm^2)$")
fig2.savefig(sys_params[0]["figure_location"]+"/init_mollweide" + sys_params[0]["plot_file_type"], dpi=300, bbox_inches='tight')


In [ ]:
aspc = 1.2
fig1 = plt.figure(num=1, figsize=(6*aspc,5), facecolor='white')
ax1 = fig1.add_axes([0.15, 0.14, 0.8, 0.8])

for ind in range(num_comparisons):
    ax1.plot(np.arange(LMAX), power_spectrum_unweighted[ind] * 100.0, linewidth=3)

ax1.set_xticks(range(0, max_mode+1, 5))
ax1.yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.1f'))
ax1.set_xlim([0, max_mode])
ax1.set_xlabel(modes_xlabel)
ax1.set_ylabel(modes_ylabel)
ax1.legend([plot_labels[0],plot_labels[1]])
fig1.savefig(sys_params[0]["figure_location"]+"/init_modes" + sys_params[0]["plot_file_type"], dpi=300, bbox_inches='tight')

## Pressure modes

In [ ]:
hs_and_modes = [None] * num_comparisons
avg_flux = [None] * num_comparisons
X_train_complex = [None] * num_comparisons
intensity_map_normalized = [None] * num_comparisons
power_spectrum = [None] * num_comparisons

LMAX = dataset_params[0]["LMAX"]

In [ ]:
for ind in range(num_comparisons):
    file_name = sys_params[ind]["heat_source_nc"]
    hs_and_modes[ind] = nrw.read_general_netcdf(run_location[ind]+"/"+file_name)

    if old_format:
        dataset_params[ind], facility_spec[ind] = tdg.define_dataset_params(int(num_examples))
    else:
        dataset_params[ind] = nrw.read_general_netcdf(diag_dir[ind] + "/" + sys_params[ind]["dataset_params_filename"])
        facility_spec[ind] = nrw.read_general_netcdf(diag_dir[ind] + "/" + sys_params[ind]["facility_spec_filename"])

    avg_flux[ind] = hs_and_modes[ind]["average_flux"][0]
    
    X_train_complex[ind] = np_complex(hs_and_modes[ind]["complex_modes"][0,:], hs_and_modes[ind]["complex_modes"][1,:])

    intensity_map_normalized[ind] = hp.alm2map(X_train_complex[ind], LMAX)

    power_spectrum[ind] = uhp.alms2power_spectrum(X_train_complex[ind], LMAX)

In [ ]:
intensity_map0 = (intensity_map_normalized[0]+1)*avg_flux[0]
intensity_map1 = (intensity_map_normalized[1]+1)*avg_flux[1]

aspc = 1.2
fig1 = plt.figure(num=1, figsize=(6*aspc,6), facecolor='white')

nside = 30
npix = hp.nside2npix(nside)
#vec = hp.ang2vec(np.pi, np.pi * 3 / 4)
vec = hp.ang2vec(np.pi/2, np.pi * 1/2)
ipix_disc = hp.query_disc(nside=nside, vec=vec, radius=np.radians(90))
masked_intensity_map = intensity_map0
masked_intensity_map[ipix_disc] = intensity_map1[ipix_disc]

projection = hp.mollview(masked_intensity_map, unit=r"$\rm{Mbar}$", title="", flip="geo", return_projected_map=True)
hp.graticule()


In [ ]:

aspc = 2.0
fig2 = plt.figure(num=2, figsize=(6*aspc,6), facecolor='white')
ax2 = fig2.add_axes([0.15, 0.14, 0.55, 0.6])
cax1 = fig2.add_axes([0.75, 0.14, 0.02, 0.6])

cmesh = ax2.pcolormesh(projection, linewidth=0, rasterized=True)
cbar = fig2.colorbar(cmesh, ax=[ax2], cax=cax1)
ax2.axis('off')

cbar.set_label("Pressure " + r"$(\rm{Mbar}$)")
fig2.savefig(sys_params[0]["figure_location"]+"/pressure_mollweide" + sys_params[0]["plot_file_type"], dpi=300, bbox_inches='tight')


In [ ]:
aspc = 1.2
fig2 = plt.figure(num=2, figsize=(6*aspc,5), facecolor='white')
ax2 = fig2.add_axes([0.15, 0.14, 0.8, 0.8])

for ind in range(num_comparisons):
    ax2.plot(np.arange(LMAX), np.sqrt(power_spectrum[ind]) * 100.0, linewidth=3)

ax2.set_xticks(range(0, max_mode+1, 5))
ax2.set_xlim([0, max_mode])
ax2.set_xlabel(modes_xlabel)
ax2.set_ylabel(modes_ylabel)
ax2.legend([plot_labels[0],plot_labels[1]])
fig2.savefig(sys_params[0]["figure_location"]+"/ablation_pressure_modes" + sys_params[0]["plot_file_type"], dpi=300, bbox_inches='tight')

## Areal density modes

In [ ]:
from netCDF4 import Dataset
import numpy as np
from scipy.interpolate import griddata
import matplotlib.ticker as mticker
import copy

In [ ]:
dataset_dict = {}
#dataset_names = ["N190204_003_cbet", "Optimized_517_rescaled"]
dataset_names = ["/mnt/local/dbarlow/solid_target/2305a_fixed_defocus/230601a_N190204_003_cbet/picked_data",
                 "/mnt/local/dbarlow/solid_target/2305a_fixed_defocus/2307c_new_laser_pulses_for_hydro/time_varying_pulse_input_deck_rescaled"]
num_datasets = len(dataset_names)

plot_time = [10.0] #ns
num_times = len(plot_time)
time_index_offset = 0

In [ ]:
for dir_name in dataset_names:
    nc = Dataset(dir_name + "/" + 'dump31_1davg.nc')
    data = nc.groups['data']

    dataset_dict[dir_name] = {}

    dataset_dict[dir_name]["time"] = data.variables["time"][:]
    dataset_dict[dir_name]["x1c"]  = data.variables["x1c"][:,:]
    dens = data.variables["dens"][:,:]
    te = data.variables["Te"][:,:]
    dataset_dict[dir_name]["lm"] = data.variables["lm"][:,:]
    dataset_dict[dir_name]["sph"] = data.variables["sph"][:,:]
    dataset_dict[dir_name]["Pe"] = data.variables["Pe"][:,:]
    dataset_dict[dir_name]["Pi"] = data.variables["Pi"][:,:]

    nc.close()

    dataset_dict[dir_name]["lm"][:,0] = 0.0
    dataset_dict[dir_name]["average_rhor"] = copy.copy(dataset_dict[dir_name]["sph"][:,0])
    dataset_dict[dir_name]["sph"][:,0] = 0.0

    dataset_dict[dir_name]["plot_time_index"] = np.array([0]*num_times)

    for ind in range(num_times):
        dataset_dict[dir_name]["plot_time_index"][ind] = np.argmin(np.abs(dataset_dict[dir_name]["time"] - plot_time[ind]))
        print(plot_time[ind], dataset_dict[dir_name]["plot_time_index"][ind])
        print(np.max(dataset_dict[dir_name]["Pe"][ind,:]), np.max(dataset_dict[dir_name]["Pi"][ind,:]))


In [ ]:
max_rhor = np.zeros((num_datasets))
counter = 0

for ind in range(num_times): 
    aspc = 1.2
    fig3 = plt.figure(num=1, figsize=(6*aspc,5), facecolor='white')
    ax3 = fig3.add_axes([0.15, 0.14, 0.8, 0.8])
    fig4 = plt.figure()
    ax4 = fig4.add_subplot(111)

    for dir_name in dataset_names:
        ind_rhor = dataset_dict[dir_name]["plot_time_index"][ind]
        print(ind_rhor)
        time = dataset_dict[dir_name]["time"]
        average_rhor = dataset_dict[dir_name]["average_rhor"]
        sph = dataset_dict[dir_name]["sph"]
        lm = dataset_dict[dir_name]["lm"]
        print("The average areal density at time ", time[ind_rhor], "ns is ", average_rhor[ind_rhor])

        ax3.plot(lm[ind_rhor,:], 100*sph[ind_rhor,:]/average_rhor[ind_rhor], linewidth=3)
        max_rhor[counter] = 100*max(sph[ind_rhor,:max_mode]/average_rhor[ind_rhor])
        print(np.argmax(sph[ind_rhor,:]/average_rhor[ind_rhor]))

        # plot pressure to check timing
        Pi = dataset_dict[dir_name]["Pi"]
        Pe = dataset_dict[dir_name]["Pe"]
        x1c = dataset_dict[dir_name]["x1c"]
        total_pressure = (Pi[ind_rhor,:] + Pe[ind_rhor,:]) / 1.0e12 # Convert to Mbar, Check!!
        ax4.plot(x1c[ind_rhor,:], total_pressure)
        counter += 1

    ax3.set_xticks(range(0, max_mode+1, 5))
    ax3.set_ylim(0, np.max(max_rhor)*1.1)
    ax3.set_xlim([0, max_mode])
    ax3.set_xlabel(modes_xlabel)
    ax3.set_ylabel(modes_ylabel)
    ax3.legend([plot_labels[0],plot_labels[1]])

    fig3.savefig("plots/rms_rhoR_at_t_"+str(int(time[ind_rhor]))+"ns.pdf")
    fig4.savefig("plots/pressure_at_t_"+str(int(time[ind_rhor]))+"ns.pdf")

## Density plot and gated X-ray

In [ ]:
num_comparison = 2

path = [None] * num_comparison
filename = [None] * num_comparison
mesh_x = [None] * num_comparison
mesh_y = [None] * num_comparison
var_data = [None] * num_comparison
time = [None] * num_comparison
cmesh = [None] * num_comparison

path[0] = "/mnt/local/dbarlow/solid_target/2305a_fixed_defocus/2307c_new_laser_pulses_for_hydro/time_varying_pulse_input_deck_rescaled/2d_slices"
filename[0] = "dens2d_10001"#'dens2d_00058.img' 
path[1] = "/mnt/local/dbarlow/solid_target/2305a_fixed_defocus/230601a_N190204_003_cbet/all_data/2d_slices"
filename[1] = "dens2d_10001"#'dens2d_00058.img'

In [ ]:
for ind in range(num_comparison):
    with open(path[ind] + "/" + filename[ind] + ".bov") as f:
        for line in f:
            if "TIME" in line:
                time[ind] = int(float(line.split(" ")[1][:-1]))
            if "VARIABLE" in line:
                var_name = line.split(" ")[1][:-1]
            if "DATA_SIZE" in line:
                #print(line.split(" "))
                x_ncells = int(line.split(" ")[1])
                y_ncells = int(line.split(" ")[2])
                print(x_ncells, y_ncells)
            if "BRICK_ORIGIN" in line:
                #print(line.split(" "))
                xmin = float(line.split(" ")[1])
                ymin = float(line.split(" ")[2])
                print(xmin, ymin)
            if "BRICK_SIZE" in line:
                #print(line.split(" "))
                xextent = float(line.split(" ")[2])
                yextent = float(line.split(" ")[4])
    xmax = xextent + xmin
    x = np.linspace(xmin, xmax, x_ncells)
    ymax = yextent + ymin
    y = np.linspace(ymin, ymax, y_ncells)

    mesh_x[ind], mesh_y[ind] = np.meshgrid(x, y)

    with open(path[ind] + "/" + filename[ind] + ".img", "rb") as f:
        # Seek backwards from end of file by 2 bytes per pixel 
        f.seek(-x_ncells*y_ncells*4, 2)
        var_data[ind] = np.fromfile(f, dtype=np.single).reshape((x_ncells, y_ncells))

In [ ]:
cut_ind = 400
combined_var_data = np.zeros((np.shape(var_data[0])))
combined_var_data[:,:cut_ind] = var_data[0][:,:cut_ind]
combined_var_data[:,cut_ind:] = var_data[1][:,cut_ind:]

aspc = 1.2
fig1 = plt.figure(num=1, figsize=(6*aspc,6), facecolor='white')
ax1 = fig1.add_axes([0.15, 0.14, 0.6, 0.6])
ax1.set_aspect('equal', adjustable='box')
cax1 = fig1.add_axes([0.8, 0.14, 0.05, 0.6])

cmesh = ax1.pcolormesh(mesh_x[ind], mesh_y[ind], combined_var_data, linewidth=0, rasterized=True)
ax1.plot([0,0],[ymin,ymax],"k", linewidth=0.5)
cbar = fig1.colorbar(cmesh, ax=[ax1], cax=cax1)

ax1.set_xlabel(r"Distance $(\mu m)$")
ax1.set_ylabel(r"Distance $(\mu m)$")
cbar.set_label(var_name + r" $(g/cm^3)$")
fig1.savefig("plots/density_comparison_"+str(int(time[0]))+"ns.pdf")
